In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential #Initialise our neural network model as a sequential network
from tensorflow.keras.layers import Conv2D #Convolution operation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Activation#Applies activation function
from tensorflow.keras.layers import Dropout#Prevents overfitting by randomly converting few outputs to zero
from tensorflow.keras.layers import MaxPooling2D # Maxpooling function
from tensorflow.keras.layers import Flatten # Converting 2D arrays into a 1D linear vector
from tensorflow.keras.layers import Dense # Regular fully connected neural network
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score

In [2]:
def load_data(dataset_path):
  
  #classes = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprsie', 'Neutral']  #We will be dealing with seven different types of emotions.

  data = []
  test_data = []
  test_labels = []
  labels =[]

  with open(dataset_path, 'r') as file:
      for line_no, line in enumerate(file.readlines()):
          if 0 < line_no <= 35887:
            curr_class, line, set_type = line.split(',')
            image_data = np.asarray([int(x) for x in line.split()]).reshape(48, 48)#Creating a list out of the string then converting it into a 2-Dimensional numpy array.
            image_data =image_data.astype(np.uint8)/255.0
            
            if (set_type.strip() == 'PrivateTest'):
              
              test_data.append(image_data)
              test_labels.append(curr_class)
            else:
              data.append(image_data)
              labels.append(curr_class)
      
      test_data = np.expand_dims(test_data, -1)
      test_labels = to_categorical(test_labels, num_classes = 7)
      data = np.expand_dims(data, -1)   
      labels = to_categorical(labels, num_classes = 7)
    
      return np.array(data), np.array(labels), np.array(test_data), np.array(test_labels)


[WinError 161] The specified path is invalid: '/\\~MTSU'
C:\Users\brice\My Drive


In [31]:
dataset_path = "~MTSU/csci4850-NeuralNets/proj/archive (2)/fer2013.csv" 
train_data, train_labels, test_data, test_labels = load_data(dataset_path)
#train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = test_size,random_state = seed)

print("Number of images in Training set:", len(train_data))
print("Number of images in Test set:", len(test_data))

Number of images in Training set: 32298
Number of images in Test set: 3589


In [32]:
#######HYPERPARAMATERS###########
epochs = 100
batch_size = 64
learning_rate = 0.001
#################################

model = Sequential()
    
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1), kernel_regularizer=l2(0.01)))
model.add(Conv2D(64, (3, 3), padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))
    
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
    
#model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
#model.add(BatchNormalization())
#model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
#model.add(BatchNormalization())
#model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
#odel.add(BatchNormalization())
#model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.5))
    
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.5))
    
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))
adam = optimizers.Adam(learning_rate = learning_rate)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
print(model.summary())

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=6, mode='auto')
checkpointer = ModelCheckpoint('/content/My Drive/Colab Notebooks/Emotion Recognition/Model/weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 46, 46, 64)        36928     
                                                                 
 batch_normalization (BatchN  (None, 46, 46, 64)       256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 23, 23, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 23, 23, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 23, 23, 64)        3

In [34]:
history = model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.2,
          shuffle = True,
#          callbacks=[lr_reducer, checkpointer, early_stopper]
          callbacks=[lr_reducer, checkpointer]

#          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/100
404/404 [==============================] - ETA: 0s - loss: 1.9506 - accuracy: 0.2076
Epoch 1: val_loss improved from inf to 1.84404, saving model to /content/My Drive/Colab Notebooks/Emotion Recognition/Model\weights.hd5
INFO:tensorflow:Assets written to: /content/My Drive/Colab Notebooks/Emotion Recognition/Model\weights.hd5\assets
404/404 [==============================] - 186s 460ms/step - loss: 1.9506 - accuracy: 0.2076 - val_loss: 1.8440 - val_accuracy: 0.2489 - lr: 0.0010
Epoch 2/100
404/404 [==============================] - ETA: 0s - loss: 1.8564 - accuracy: 0.2374
Epoch 2: val_loss improved from 1.84404 to 1.84207, saving model to /content/My Drive/Colab Notebooks/Emotion Recognition/Model\weights.hd5
INFO:tensorflow:Assets written to: /content/My Drive/Colab Notebooks/Emotion Recognition/Model\weights.hd5\assets
404/404 [==============================] - 185s 457ms/step - loss: 1.8564 - accuracy: 0.2374 - val_loss: 1.8421 - val_accuracy: 0.2489 - lr: 0.0010
Epoch 

In [35]:
predicted_test_labels = np.argmax(model.predict(test_data), axis=1)
test_labels = np.argmax(test_labels, axis=1)
print ("Accuracy score = ", accuracy_score(test_labels, predicted_test_labels))

Accuracy score =  0.6140986347171914


In [36]:
from tensorflow.keras.models import model_from_json
model_json = model.to_json()
with open("/content/gdrive/My Drive/Colab Notebooks/Emotion Recognition/model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/My Drive/Colab Notebooks/Emotion Recognition/model.h5")
print("Saved model to disk")

Saved model to disk


In [41]:
epochs = 10
batch_size = 64

In [42]:
history = model.fit(
          train_data,
          train_labels,
          epochs = epochs,
          batch_size = batch_size,
          validation_split = 0.2,
          shuffle = True,
#          callbacks=[lr_reducer, checkpointer, early_stopper]
          callbacks=[lr_reducer, checkpointer]

#          callbacks=[lr_reducer, early_stopper]
          )

Epoch 1/10
404/404 [==============================] - ETA: 0s - loss: 0.9678 - accuracy: 0.6452
Epoch 1: val_loss did not improve from 1.11792
404/404 [==============================] - 257s 636ms/step - loss: 0.9678 - accuracy: 0.6452 - val_loss: 1.1217 - val_accuracy: 0.6176 - lr: 2.2877e-04
Epoch 2/10
404/404 [==============================] - ETA: 0s - loss: 0.9604 - accuracy: 0.6468
Epoch 2: val_loss did not improve from 1.11792
404/404 [==============================] - 278s 688ms/step - loss: 0.9604 - accuracy: 0.6468 - val_loss: 1.1201 - val_accuracy: 0.6153 - lr: 2.2877e-04
Epoch 3/10
404/404 [==============================] - ETA: 0s - loss: 0.9517 - accuracy: 0.6522
Epoch 3: val_loss improved from 1.11792 to 1.11555, saving model to /content/My Drive/Colab Notebooks/Emotion Recognition/Model\weights.hd5
INFO:tensorflow:Assets written to: /content/My Drive/Colab Notebooks/Emotion Recognition/Model\weights.hd5\assets
404/404 [==============================] - 287s 711ms/step -

In [43]:
from keras.models import model_from_json
import numpy as np
import cv2

def load_model(path):

	json_file = open(path + 'model.json', 'r')
	loaded_model_json = json_file.read()
	json_file.close()
	
	model = model_from_json(loaded_model_json)
	model.load_weights(path + "model.h5")
	print("Loaded model from disk")
	return model
	
def predict_emotion(gray, x, y, w, h):
	face = np.expand_dims(np.expand_dims(np.resize(gray[y:y+w, x:x+h]/255.0, (48, 48)),-1), 0)
	prediction = model.predict([face])

	return(int(np.argmax(prediction)), round(max(prediction[0])*100, 2))
	
path = "Models/"
model = load_model(path)

fcc_path = "Tools/haarcascade_frontalface_alt.xml"
faceCascade = cv2.CascadeClassifier(fcc_path)
emotion_dict = {0: "Angry", 1: "Disgust", 2: "Fear", 3: "Happy", 4: "Sad", 5: "Surprise", 6: "Neutral"}
colour_cycle = ((255, 0, 0), (0, 255, 0), (0, 0, 255), (230, 230, 250))
webcam = cv2.VideoCapture(0)

while True:
	ret, frame = webcam.read()
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
	
	faces = faceCascade.detectMultiScale(
			gray,
			scaleFactor=1.1,
			minNeighbors=5,
			minSize=(30, 30)
									)
								
	for (count,(x, y, w, h)) in enumerate(faces):
		colour = colour_cycle[int(count%len(colour_cycle))]
		cv2.rectangle(frame, (x, y), (x+w, y+h), colour, 2)
		cv2.line(frame, (x+5, y+h+5),(x+100, y+h+5), colour, 20)
		cv2.putText(frame, "Face #"+str(count+1), (x+5, y+h+11), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), lineType=cv2.LINE_AA)

		cv2.line(frame, (x+8, y),(x+150, y), colour, 20)
		emotion_id, confidence = predict_emotion(gray, x, y, w, h)
		emotion = emotion_dict[emotion_id]
		cv2.putText(frame, emotion + ": " + str(confidence) + "%" , (x+20, y+5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), lineType=cv2.LINE_AA)
	
	cv2.imshow('Emotion Recognition - Press q to exit.', frame)
	if cv2.waitKey(1) & 0xFF == ord('q'): break

webcam.release()
cv2.destroyAllWindows()

FileNotFoundError: [Errno 2] No such file or directory: 'Models/model.json'